In [1]:
// set up dependencies
// use local maven repository; not yet deployed to remote maven repositories.
@file:Repository("*mavenLocal")
@file:Repository("https://maven.scijava.org/content/groups/public")
@file:Repository("https://jitpack.io")

// uncomment to search in your local maven repo
// requires installation into local maven repository (./gradlew build publishToMavenLocal)
@file:DependsOn("net.imglib2:imklib2:0.1.0-SNAPSHOT")

// uncomment to search in jitpack
// @file:DependsOn("com.github.saalfeldlab:imklib2:26c21b71795e20d94a90a35b4700ae0c66044bdf")

// Add BDV vistools dependency for visualization
@file:DependsOn("sc.fiji:bigdataviewer-vistools:1.0.0-beta-21")

%use lets-plot

In [2]:
import kotlin.math.sqrt

import java.nio.file.Paths

import bdv.util.BdvFunctions
import bdv.util.BdvOptions
import bdv.viewer.DisplayMode

import net.imglib2.Point
import net.imglib2.imklib.*
import net.imglib2.type.numeric.integer.UnsignedByteType

In [3]:
// only do gradient in xy
val gradientDimensions = intArrayOf(0, 1)
// anisotropic data -> anisotropic cache block size
val cacheBlockSize = intArrayOf(32, 32, 3)

Download an HDF5 data set with 3D volumetric data, e.g. one of the [CREMI](https://cremi.org/data/) data sets. Update the paths in the next cell accordingly

In [4]:
val path = Paths.get(System.getProperty("user.home"), "Downloads", "sample_A_20160501.hdf")
val rawData = imklib.io.n5.openHDF5<UnsignedByteType>("$path", "volumes/raw")
val rawDataExtended = rawData.extendBorder().asLongs()

In [5]:
val offsets = gradientDimensions.map { d -> Point(3).also{ p -> p.setPosition(1L, d) } }
val firstPartialDerivatives = offsets.map { o -> 
    val diff = (rawDataExtended + o) - (rawDataExtended - o)
    diff[rawData].cache()
}
val firstDerivativeMagnitude = firstPartialDerivatives
    .map { it * it }.reduce { d1, d2 -> d1 + d2 }
    .convert(imklib.types.double) { s, t -> t.set(sqrt(s.realDouble)) }
    .cache()

In [6]:
val bdv = BdvFunctions.show(rawData.volatileView, "raw").also { it.setDisplayRange(0.0, 255.0) }

// set up BDV
val vp = bdv.getBdvHandle().viewerPanel
vp.state().displayMode = DisplayMode.SINGLE

// add derivatives
for ((dim, derivative) in gradientDimensions zip firstPartialDerivatives) {
    BdvFunctions
        .show(derivative.volatileView, "first-derivative-$dim", BdvOptions.options().addTo(bdv))
        .setDisplayRange(0.0, 100.0)
}
BdvFunctions
    .show(firstDerivativeMagnitude.volatileView, "first-derivative-magnitude", BdvOptions.options().addTo(bdv))
    .setDisplayRange(0.0, 100.0)